In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GitHub" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.

# Install dependencies
# !pip install wget
# !apt-get install sox libsndfile1 ffmpeg
# !pip install text-unidecode
# !pip install matplotlib>=3.3.2


## Install NeMo
BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
# !mkdir configs
# !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/contextnet_rnnt/contextnet_rnnt.yaml

In [2]:
import os
import sys


# Insert local paths at the beginning of sys.path
sys.path.insert(0, os.path.abspath('/workspace/nemo/NeMo-opensource/'))

import nemo.collections.asr as nemo_asr
print(nemo_asr.__file__)
import nemo.core as nemo_core
print(nemo_core.__file__)
from nemo.core import adapter_mixins
import nemo
print(nemo.__file__)
# Restore the site-packages paths
# sys.path.extend(site_packages_paths)

/workspace/nemo/NeMo-opensource/nemo/collections/asr/__init__.py
/workspace/nemo/NeMo-opensource/nemo/core/__init__.py
/workspace/nemo/NeMo-opensource/nemo/__init__.py


In [3]:
import lightning
print(lightning.__file__)

/usr/local/lib/python3.10/dist-packages/lightning/__init__.py


# ASR Domain Adaptation with Adapters

Throughout various Automatic Speech Recognition tutorials, you may have noticed that ASR datasets are generally enormous - on the order of hundreds or even thousands of hours of speech. Such a large amount of data imposes significant restrictions on the development of ASR models due to the cost of collection of labeled data (or even unlabeled datasets) and the severe compute cost to train a model on that much data. 

We further see that when fine-tuning a pre-trained model, we need large datasets and compute to achieve superior results and avoid overfitting to the new dataset. Worse, by training the entire model (or even just the decoder modules), we severely degrade the model's performance on the original dataset.

-----

In this tutorial, we will showcase **Adapters** : A powerful method to efficiently adapt a pre-trained model to a new dataset (with minimal amounts of data, even just 30 minutes !) with minimal compute resources (on a single GPU, in around 10 minutes of training time).


## What are Adapters?

Adapters are a straightforward concept proposed in multiple papers across various domains. For the sake of brevity, we will choose a recent paper [Using Adapters to Overcome Catastrophic Forgetting in End-to-End Automatic Speech Recognition](https://arxiv.org/abs/2203.16082) as a reference for this discussion.

Essentially, an **Adapter** is any **trainable module that is added * after * a model has been trained to convergence**. 

- These additional modules form a residual bridge over the output of each layer they adapt, such that the model's original performance is not lost. 
- The original parameters of the model are frozen in their entirety - so that we don't lose performance on the original domain.
- We train only the new adapter parameters (an insignificant fraction of the total number of parameters). This allows fast experimentation.

-----

Adapters are a straightforward concept - as shown by the diagram below. At their simplest, they are residual Feedforward layers that compress the input dimension ($D$) to a small bottleneck dimension ($H$), such that $R^D \text{->} R^H$, compute an activation (such as ReLU), finally mapping $R^H \text{->} R^D$ with another Feedforward layer. This output is then added to the input via a simple residual connection.

<div align="center">
  <img src="https://mermaid.ink/img/pako:eNptkLFqwzAQhl9F3ORAPDSjA4EUx6RgXEjbycpwWOdG1JaMfEoakrx7ZcfpUKrlxH_fz4d0gcoqggTqxp6qAzoW76k0Ipx1-WI6z3sRxyuRF1GOZ3KisK6d3YG8GFdZ9hRJeLbMDRmqvkRGpDLrTuiUiEWUigBtlyIVqzBnEqZ66I39dcX6iKytKXeUf-wn-286QoFeBMvmu0PTD-EfyXaQpP9JFmP_1XN4S3kfD8W4ue6o18pjc52gYQlzaMm1qFX4msuQSOADtSQhCdfaOupZgjS3QPpOIdNGabYOkhqbnuaAnu3b2VSQsPP0gFKNnw7bibr9AJkZdXU" height=100% />
</div>

-----

Adapter modules such as this are usually initialized. The initial output of the adapter will always be zeros to prevent degradation of the original model's performance due to the addition of such modules.

## Advantages of Adapters

Since adapters are additional parameters added to an already trained network, and due to their construction, they possess multiple beneficial properties.

- **Added parameter cost**: They cost an insignificant number of parameters compared to the original model (generally 0.5% - 1% of the initial parameter count).
- **Residual bridge**: Adapters are initialized with special care, such that their contribution to the output of each layer they adapt is initially 0. Therefore, after the addition of the adapters, the original model does not lose any accuracy at all (even without training the adapters).
- **Fast convergence**: Since the adapters only need to learn to modify the module's output slightly, and each adapter has a trivial parameter cost, they converge rapidly.
- **Adapt only the encoder**: Adapters can be used anywhere, but they are most commonly used in just the encoder, keeping the decoder modules frozen. This allows the decoder to be unaffected by costly CTC/RNN-T training, which takes time to converge, and just the adapter modules in the encoder need to be updated.
- **Dynamic and flexible adaptation**: Since adapter modules can be added any number of times, a single shared "core" model can have multiple adapters that are enabled/disabled dynamically to adapt to numerous scenarios. This potentially offers the case where a single "core" model is shared across multiple users, and each user has a small, personal adapter module used for personalization.  

## Limitations of Adapters

With all those benefits, Adapters have significant drawbacks stemming from how they are currently used.

Note that there is ongoing research to overcome such restrictions, which we hope to incorporate eventually.

- **Frozen decoder modules**: Since the decoder is frozen to avoid forgetting the past training, it does not learn the semantics of the text from the new domain. This may hamper some models more than others.
- **Frozen tokenization/decoding**: A consequence of the frozen decoder is that we cannot change the vocabulary (or tokenizer) of the decoder layer - as this would cause the model to forget its past training entirely. This also means that **adapters cannot be used to train in a different language than the original language**. The text of the new domain must be supported by the original model's tokenizer/decoder.
  - **Note**: There is nothing fundamentally wrong with still changing the vocabulary of a model that supports adapters. The benefits of adapters will reduce significantly and require costly training (similar in time and memory to finetuning). The model can no longer recover its performance by disabling all of its adapters.
- **Easy to overfit**: Since adapters enable domain adaptation on very small amounts of speech data, it is trivial to rapidly overfit these datasets and significantly degrade performance on the original domain. 
  - **Note**: This can be overcome with some experimentation, further boosted by the fast experimentation cycle that adapters enable.

# Dataset preparation

Now that we understand what adapters are and their benefits/drawbacks, we can explore how to set up adapters in NeMo ASR models.

-----

First, we prepare some datasets that the original model was **not trained on**, making it a new domain to be adapted. 

In this tutorial, we will be utilizing the `AN4` dataset - also known as the Alphanumeric dataset, which was collected and published by Carnegie Mellon University. We chose this dataset primarily because it is **very small in size** (`<1 hours of training data`), **easy to overfit when training from scratch / fine-tuning by changing the decoder** (`previous tutorials can mostly get around 10-20% WER with fine-tuning without hyperparameter tuning`), and its **text is perfectly supported by the tokenization/decoding scheme of the model**.

In [4]:
import os

if not os.path.exists("scripts/"):
  os.makedirs("scripts")

if not os.path.exists("scripts/process_an4_data.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/dataset_processing/process_an4_data.py

In [5]:
import wget
import tarfile 
import subprocess 
import glob

data_dir = "datasets"

if not os.path.exists(data_dir):
  os.makedirs(data_dir)

# Download the dataset. This will take a few moments...
print("******")
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'


if not os.path.exists(data_dir + '/an4/'):
    # Untar and convert .sph to .wav (using sox)
    tar = tarfile.open(an4_path)
    tar.extractall(path=data_dir)

    print("Converting .sph to .wav...")
    sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
    for sph_path in sph_list:
        wav_path = sph_path[:-4] + '.wav'
        cmd = ["sox", sph_path, wav_path]
        subprocess.run(cmd)

print("Finished conversion.\n******")

if os.path.exists(f"{data_dir}/an4"):
  print("Preparing AN4 dataset ...")

  an4_path = f"{data_dir}/"
  !python scripts/process_an4_data.py \
    --data_root=$an4_path

print("AN4 prepared !")

******
Tarfile already exists.
Finished conversion.
******
Preparing AN4 dataset ...


/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/scripts/process_an4_data.py:43: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.core.get_duration(filename=audio_path)
AN4 prepared !


In [6]:
# Manifest filepaths
TRAIN_MANIFEST = os.path.join(data_dir, "an4", "train_manifest.json")
TEST_MANIFEST = os.path.join(data_dir, "an4", "test_manifest.json")

# Prepare the "base" model

Next, we initialize a small pre-trained model with a relatively small amount of parameters to showcase the efficacy of adapters no matter the original model size.

-----

Most importantly, we discuss a simple way to enable Adapter specific support to a pre-trained model checkpoint - by modifying the `encoder` config before loading the model.

In [7]:
import torch
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
wandb_logger = WandbLogger(project="NEMO_TEST")
# import nemo.collections.asr as nemo_asr

In [8]:
model_name = "stt_en_conformer_ctc_large"

## Prepare an Adapter-compatible Encoder

The original model was trained without any adapters, and therefore its encoder does not support adapters.

 To add adapter modules to these models, we perform a few simple steps - 

- Extract the model config from the "base" model.
- Update the `encoder` section of the config to a subclass of that model (which does have Adapter support)
- Initialize the model with this new config, therefore enabling adapter support.

- Extract just the config of the model.

In [9]:
cfg = nemo_asr.models.ASRModel.from_pretrained(model_name, return_config=True)

[NeMo I 2024-07-22 20:42:32 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.
[NeMo I 2024-07-22 20:42:32 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo
[NeMo I 2024-07-22 20:42:32 common:815] Instantiating model from pre-trained checkpoint


In [10]:

# Utility method to check and update the model config
def update_model_config_to_support_adapter(model_cfg):
    with open_dict(model_cfg):
        adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
        if adapter_metadata is not None:
            model_cfg.encoder._target_ = adapter_metadata.adapter_class_path
    
    print("Updated encoder _target_ model :", model_cfg.encoder._target_)
    return model_cfg

- Update the model config's `encoder` section to support Adapters.

In [11]:
cfg = update_model_config_to_support_adapter(cfg)

Updated encoder _target_ model : nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter


- Finally load the model with the updated config.

In [12]:
model = nemo_asr.models.ASRModel.from_pretrained(model_name, override_config_path=cfg)

[NeMo I 2024-07-22 20:42:32 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.
[NeMo I 2024-07-22 20:42:32 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo
[NeMo I 2024-07-22 20:42:32 common:815] Instantiating model from pre-trained checkpoint
[NeMo I 2024-07-22 20:42:33 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2024-07-22 20:42:33 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket1/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket2/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket3/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket4/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket5/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket6/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket7/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    trim_

[NeMo I 2024-07-22 20:42:33 features:305] PADDING: 0
[NeMo I 2024-07-22 20:42:34 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.


-----

Next, we attach a `Trainer` to the model to be appropriately initialized.

Note that we select just **300 update steps**, which is approximately just ten epochs over this dataset at batch sizes of 32. You can experiment with different steps to see the effect of overfitting or underfitting.

**Recommendation**:

You should normally start with 1-5 epochs of adaptation over your entire new domain, and then increase or decrease your number of training steps to trade off a balance in accuracy on general speech.

In [13]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
max_steps = 300

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=3)

model.set_trainer(trainer)

INFO: GPU available: True (cuda), used: True
I0722 20:42:51.693691 129733185169216 rank_zero.py:64] GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
I0722 20:42:51.715736 129733185169216 rank_zero.py:64] TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
I0722 20:42:51.716401 129733185169216 rank_zero.py:64] HPU available: False, using: 0 HPUs


In [14]:
# utility method
import json
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest


## [Optional] Check if the new domain is compatible with the original decoder

The following section, while optional, designs a test to ensure that the text of the new domain can be adequately handled by the original decoder/tokenizer of the model. Please open each cell and execute to perform this sanity check.

-----

If this check fails, the training run might crash, or silently allow the model to learn to produce `⁇` tokens (when using SentencePiece tokenizers).

### Parse the base character set

In [15]:
# train_data = read_manifest(TRAIN_MANIFEST)
# base_sets = [set(list(sample['text'])) for sample in train_data]
# base_charset = set([])
# for charset in base_sets:
#   base_charset.update(charset)
# base_charset = list(sorted(list(base_charset)))

# print("Base charset :", base_charset)

### Check if there are invalid characters

In [34]:
# def check_valid_charset_in_vocab(model, charset):
#   model_vocab = model.decoder.vocabulary
#   num_invalid = 0

#   for char in charset:
#     if char != ' ' and char not in model_vocab:
#       print(f"Character `{char}` does not exist in the base character set of the original model !")
#       num_invalid += 1

#   print("Number of invalid tokens :", num_invalid)

In [16]:
# check_valid_charset_in_vocab(model, base_charset)

Number of invalid tokens : 0


# Evaluate original performance on AN4 dev set

Now that we possess a model capable of supporting adapters, let us quickly test the performance of the pre-trained model on the AN4 test set without any training or fine-tuning.

In [15]:
if not os.path.exists('scripts/transcribe_speech.py'):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/transcribe_speech.py

if not os.path.exists('scripts/speech_to_text_eval.py'):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/speech_to_text_eval.py

In [18]:
# # temporarily save current model
# model.save_to("/content/unadapted_model.nemo")

-----

The following evaluation script will properly transcribe the AN4 test set, and score it against its ground truth.

In [19]:
# !python scripts/speech_to_text_eval.py \
#   model_path="/content/unadapted_model.nemo" \
#   dataset_manifest=$TEST_MANIFEST \
#   output_filename="/content/unadapted_predictions.json" \
#   batch_size=32 \
#   use_cer=False

Traceback (most recent call last):
  File "/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/scripts/speech_to_text_eval.py", line 71, in <module>
    import transcribe_speech
  File "/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/scripts/transcribe_speech.py", line 29, in <module>
    from nemo.collections.asr.models.aed_multitask_models import parse_multitask_prompt
ImportError: cannot import name 'parse_multitask_prompt' from 'nemo.collections.asr.models.aed_multitask_models' (/usr/local/lib/python3.10/dist-packages/nemo/collections/asr/models/aed_multitask_models.py)


------

Check the predictions of the current model

In [20]:
# !head -n 5 /content/unadapted_predictions.json

head: cannot open '/content/unadapted_predictions.json' for reading: No such file or directory


-----

Overall, the model does quite well, obtaining roughly 6% Word Error Rate without prior training on this dataset. 

**Note**: Pre-trained models in NeMo are trained on several thousands of hours of speech, so it is unsurprising why this model is this accurate without any training on this toy dataset. For more realistic cases, we usually observe the range of 10-30% WER for out-of-domain speech.

# Setup training and evaluation of the model

Now that we have a baseline result, let us set up the data loaders of this model to prepare for training on this dataset.

You may note: this step is nearly identical to from scratch training / fine-tuning and skips the tokenizer construction/change vocabulary steps.

**Note**: Each model may have special parameters in their data loader. Please refer to the configs of the pre-trained models to determine what additional changes are necessary). Below recommendations are primarily for Conformer CTC and may differ from model to model.

You can parse the model config via - `print(OmegaConf.to_yaml(model.cfg))`

## Setup dataloaders

In [47]:
with open_dict(model.cfg):
  # Train Dataloader
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.batch_size = 32
  model.cfg.train_ds.is_tarred = False
  model.cfg.train_ds.tarred_audio_filepaths = None

  model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
  model.cfg.validation_ds.batch_size = 32

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)

[NeMo I 2024-07-19 09:32:48 collections:199] Dataset loaded with 948 files totalling 0.71 hours
[NeMo I 2024-07-19 09:32:48 collections:201] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-19 09:32:48 collections:199] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2024-07-19 09:32:48 collections:201] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-19 09:32:48 collections:199] Dataset loaded with 130 files totalling 0.10 hours
[NeMo I 2024-07-19 09:32:48 collections:201] 0 files were filtered totalling 0.00 hours


## Setup Spectrogram Augmentation

For this experiment we will continue to use the original spec augmentation config in the base model, however you may find better results by modifying the strength of this augmentation.

**Note**: The script inside ASR examples **disables spec augment entirely**. This is done in order to provide a stable default to measure the best possible adaptation case, but may severely degrade the performance on general speech. Please be careful when copying the hyper parameters from the tutorial to the script for large scale experimentation.

In [48]:
for key in model.cfg.keys():
  print(f"{key} : {model.cfg[key]}")

sample_rate : 16000
log_prediction : True
ctc_reduction : mean_batch
skip_nan_grad : False
train_ds : {'manifest_filepath': 'datasets/an4/train_manifest.json', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 4, 'pin_memory': True, 'use_start_end_token': False, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': [34, 30, 26, 22, 18, 16, 12, 8]}
validation_ds : {'manifest_filepath': 'datasets/an4/test_manifest.json', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False}
test_ds : {'manifest_filepath': 'datasets/an4/test_manifest.json', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False}
tokenizer : {'dir': '/tokenizers/NeMo_ASR_SET/English/asr_set_3.0/tokenizer_spe_unigra

In [49]:
with open_dict(model.cfg):
  # Spec Augment
  model.cfg.spec_augment.freq_masks = model.cfg.spec_augment.freq_masks  # Can be changed
  model.cfg.spec_augment.freq_width = model.cfg.spec_augment.freq_width  # Can be changed
  model.cfg.spec_augment.time_masks = model.cfg.spec_augment.time_masks  # Can be changed
  model.cfg.spec_augment.time_width = model.cfg.spec_augment.time_width  # Can be changed

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

## Setup optimizer and scheduler

An interesting thing to note for adapters is their rapid convergence speed - they do not require hundreds of thousands of update steps (though that is also possible).

For this reason, we have chosen hyperparameter settings that are significantly different from other tutorials - a small learning rate multiplier of 0.1 (for NoamScheduler) and a small warmup phase of just 100 steps (remember, trainer.max_steps is just 300!).

Feel free to modify these values to see the effect on adapters' convergence.

**Note**: The hyper parameters below correspond to the base model and may not match those applied in the ASR examples! Please note that the script the examples defaults to an **AdamW** optimizer with a **CosineAnnealing** scheduler, where as the config of Conformers is geneally a **AdamW** optimizer with a **NoamAnnealing** scheduler. The *learning rate*, *weight decay* and other hyper parameters may not be exactly the same between the tutorial and the example scripts, so please be careful when transferring the hyper parameters for large scale experiments.

In [50]:
if 'optim' in model.cfg:
  print(OmegaConf.to_yaml(model.cfg.optim))

name: adamw
lr: 2.0
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: NoamAnnealing
  d_model: 512
  warmup_steps: 10000
  warmup_ratio: null
  min_lr: 1.0e-06



In [51]:
with open_dict(model.cfg):
  model.cfg.optim.lr = 0.1
  model.cfg.optim.weight_decay = 0.0
  model.cfg.optim.sched.warmup_steps = 100

model.setup_optimization(model.cfg.optim);

[NeMo I 2024-07-19 09:32:52 modelPT:767] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-19 09:32:52 lr_scheduler:923] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x7eb68c4f1210>" 
    will be used during training (effective maximum steps = 300) - 
    Parameters : 
    (d_model: 512
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 300
    )


# Adapters: Supported Components

A NeMo model may have multiple types of adapters that are supported in each of their components. Let us see at a glance what are some of the adapter types supported by the Conformer ASR model.

**Note**: Every domain may support their own types of adapters, and use them in different ways. Please refer to the documentation of each domain for information on the adapter support.

-----
Let's start with the modules in which the model will support adapters. We can select these adapters with a special syntax to construct "Module adapters".

**Note**: `''` refers to the "default" adapter - usually the `encoder` but it is model dependent. It may also be that no specific modules are provided, in which case only `default` adapters will be available.

In [53]:
if hasattr(model, 'adapter_module_names'):
  print(model.adapter_module_names)

['', 'encoder', 'decoder', 'joint']


-----
Next, we can try to obtain the accepted types of each of the child modules in the Model.

In [54]:
print(model.children())
for module in model.children():
  if hasattr(module, 'get_accepted_adapter_types'):
    types = module.get_accepted_adapter_types()
    print("Module : ", module.__class__.__name__)

    for tp in types:
      print(tp)
    print()

<generator object Module.children at 0x7eb68d26be60>
Module :  ConformerEncoderAdapter
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.MultiHeadAttentionAdapter'>
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.RelPositionMultiHeadAttentionAdapter'>
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>

Module :  ConvASRDecoder
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>



-----

As you can see, a single component of the model may support one or more adapter types (or none at all)! Below, we will experiment with the simple Linear Adapters, but as an exercise, you might try to use other adapter types present here.

# Adapters: Creation and Preparation

Now that the data loaders have been prepared, the next step is to add the adapter modules to the model!

----

We first import a config for a basic `LinearAdapter` most often used in literature. 

`LinearAdapter` is a simple network comprising LayerNorm, a bottleneck Linear layer, an activation, and an upcast Linear layer (so that input and output channel dim match). We provide some configuration parameters (such as the input dim and the bottleneck dim).

In [55]:
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig
from nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module import MultiHeadAttentionAdapterConfig
from nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module import RelPositionMultiHeadAttentionAdapterConfig

In [58]:
#%% [code]
#@title Adapter Setup { display-mode: "form" }
adapter_name = "AN4" #@param {type:"string"}
adapter_dim = 64 #@param {type:"integer"}
adapter_activation = "swish" #@param {type:"string"}
adapter_norm_position = "pre" #@param ["pre", "post"]

In [59]:
adapter_cfg = LinearAdapterConfig(
    in_features=model.cfg.encoder.d_model,  # conformer specific model dim. Every layer emits this dim at its output.
    dim=adapter_dim,  # the bottleneck dimension of the adapter
    activation=adapter_activation,  # activation used in bottleneck block
    norm_position=adapter_norm_position,  # whether to use LayerNorm at the beginning or the end of the adapter
)
print(adapter_cfg)

LinearAdapterConfig(in_features=512, dim=64, activation='swish', norm_position='pre', dropout=0.0, adapter_strategy=ResidualAddAdapterStrategyConfig(stochastic_depth=0.0, l2_lambda=0.0, _target_='nemo.core.classes.mixins.adapter_mixin_strategies.ResidualAddAdapterStrategy'), _target_='nemo.collections.common.parts.adapter_modules.LinearAdapter')


## Add a new adapter module

Now that our adapter config is ready. Next, we perform a check to see what is the size of the original model and what its size will be after adding the adapter module.

In [60]:
model.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoderAdapter           | 121 M  | train
2 | decoder           | ConvASRDecoder                    | 66.2 K | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
121 M     Trainable params
0         Non-trainable params
121 M     Total params
486.005   Total estimated model params size (MB)

-----

Next, we use `add_adapter` to add adapter blocks to the `encoder`.

A single line can be used to add adapter modules to every layer of the `encoder` module. We pass it a unique name to identify this adapter and the adapter config (which can be helpful to enable or disable adapters later).

In [61]:
model.add_adapter(name=adapter_name, cfg=adapter_cfg)

-----

As expected, the number of parameters increased by a marginal amount (roughly 200,000 parameters).

In [62]:
model.summarize()

  | Name              | Type                              | Params | Mode 
--------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | train
1 | encoder           | ConformerEncoderAdapter           | 122 M  | train
2 | decoder           | ConvASRDecoder                    | 66.2 K | train
3 | loss              | CTCLoss                           | 0      | train
4 | spec_augmentation | SpectrogramAugmentation           | 0      | train
5 | wer               | WER                               | 0      | train
--------------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
490.798   Total estimated model params size (MB)

## Enable / Disable Adapters

Now that we have adapter modules, we can enable or disable them as we require. 

For this purpose, we utilize the `model.set_enabled_adapters` method - it takes an optional `name` and a boolean value for `enabled`. If a name is not passed, it will set enable/disable all available adapters.

**Note**: We recommend training one adapter at a time, disjoint from all other adapters. As such, it simplifies the selection of adapters for each particular domain. To do so - **disable all adapters first, then enable only the newly added adapter**.

In [63]:
model.set_enabled_adapters(enabled=False)  # disable all adapters
model.set_enabled_adapters(name=adapter_name, enabled=True)  # enable only the current adapter we want to train

[NeMo I 2024-07-19 09:39:28 adapter_mixins:719] Setting adapter 'AN4' status : Enabled = False
[NeMo I 2024-07-19 09:39:28 adapter_mixins:734] Setting adapter 'AN4' status : Enabled = True


## Training only the adapter(s)

Now that we have enabled just the adapter we wish to adapt onto AN4 dataset, we must freeze all the other parameters of the network and train just the adapters.

We provide the general utility methods for this purpose - `model.freeze()` and `model.unfreeze_enabled_adapters()`. 

The second method will look up all the enabled adapters selected in the previous step and enable their gradient calculation so that they can be trained.

In [64]:
model.freeze()
model.unfreeze_enabled_adapters()

[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.1.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.2.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.3.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.4.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-19 09:39:34 adapter_mixins:405] Froze module encoder.layers.5.conv.batch_norm: BatchNorm1d(512, eps

### Why are BatchNormalization layers being frozen?

A side-note here regarding BatchNormalization - even when the model is frozen, while no gradient updates will occur to the beta and gamma parameters, **the moving averages of each batch norm layer continue to update**!

Such updates cause a severe issue when we disable all the adapters - while technically, no parameters of the original model were updated since the moving averages of BatchNormalization were updated. You will have degraded performance on the original domain (even with all adapters disabled !).

-----

For this reason, `unfreeze_enabled_adapters()` has an argument `freeze_batchnorm=True` as the default. It will find all the batch normalization layers and disable this flag so that it will the encoder layers remain exactly frozen even during adapter finetuning. This allows the original model performance to be recovered.



In [65]:
model.summarize()

  | Name              | Type                              | Params | Mode
-------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | eval
1 | encoder           | ConformerEncoderAdapter           | 122 M  | eval
2 | decoder           | ConvASRDecoder                    | 66.2 K | eval
3 | loss              | CTCLoss                           | 0      | eval
4 | spec_augmentation | SpectrogramAugmentation           | 0      | eval
5 | wer               | WER                               | 0      | eval
-------------------------------------------------------------------------------
1.2 M     Trainable params
121 M     Non-trainable params
122 M     Total params
490.798   Total estimated model params size (MB)

-----

Here we see that after the above steps, we will be training just ~ 200,000 parameters out of a 10+ M parameter model.

In [31]:
# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager


# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/',
    name=f"ASR-Adapters",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
    create_wandb_logger=True,
    wandb_logger_kwargs=OmegaConf.create({"project": "NEMO_TEST", "name": "ASR-Adapters", "log_model":"all"}),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2024-07-08 10:06:22 exp_manager:396] Experiments will be logged at experiments/ASR-Adapters/2024-07-08_10-06-22
[NeMo I 2024-07-08 10:06:22 exp_manager:856] TensorboardLogger has been set up
[NeMo I 2024-07-08 10:06:22 exp_manager:871] WandBLogger has been set up


[NeMo W 2024-07-08 10:06:22 exp_manager:966] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 300. Please ensure that max_steps will run for at least 3 epochs to ensure that checkpointing will not error out.


In [33]:
# Finally, train the adapters
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-07-07 13:27:08 modelPT:767] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-07 13:27:08 lr_scheduler:923] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x77979d4efc70>" 
    will be used during training (effective maximum steps = 300) - 
    Parameters : 
    (d_model: 176
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 300
    )



  | Name              | Type                              | Params | Mode
-------------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0      | eval
1 | encoder           | ConformerEncoderAdapter           | 13.2 M | eval
2 | decoder           | ConvASRDecoder                    | 181 K  | eval
3 | loss              | CTCLoss                           | 0      | eval
4 | spec_augmentation | SpectrogramAugmentation           | 0      | eval
5 | wer               | WER                               | 0      | eval
-------------------------------------------------------------------------------
185 K     Trainable params
13.2 M    Non-trainable params
13.3 M    Total params
53.360    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-07-07 13:27:08 wer:334] 
    
[NeMo I 2024-07-07 13:27:08 wer:335] reference:rubout g m e f three nine
[NeMo I 2024-07-07 13:27:08 wer:336] predicted:rabutt g m e f three nine
[NeMo I 2024-07-07 13:27:08 wer:334] 
    
[NeMo I 2024-07-07 13:27:08 wer:335] reference:v a n e s s a
[NeMo I 2024-07-07 13:27:08 wer:336] predicted:v a n e s s a


...

Validation: |          | 0/? [00:00<?, ?it/s]

[NeMo I 2024-07-07 13:27:50 wer:334] 
    
[NeMo I 2024-07-07 13:27:50 wer:335] reference:rubout g m e f three nine
[NeMo I 2024-07-07 13:27:50 wer:336] predicted:rubout g m e f three nine
[NeMo I 2024-07-07 13:27:51 wer:334] 
    
[NeMo I 2024-07-07 13:27:51 wer:335] reference:v a n e s s a
[NeMo I 2024-07-07 13:27:51 wer:336] predicted:v a n e s s a
[NeMo I 2024-07-07 13:27:51 wer:334] 
    
[NeMo I 2024-07-07 13:27:51 wer:335] reference:march seven nineteen sixty seven
[NeMo I 2024-07-07 13:27:51 wer:336] predicted:march seven nineteen sixty seven
[NeMo I 2024-07-07 13:27:51 wer:334] 
    
[NeMo I 2024-07-07 13:27:51 wer:335] reference:m y e r s
[NeMo I 2024-07-07 13:27:51 wer:336] predicted:m y e r s
[NeMo I 2024-07-07 13:27:51 wer:334] 
    
[NeMo I 2024-07-07 13:27:51 wer:335] reference:four one two two six eight four one four two
[NeMo I 2024-07-07 13:27:51 wer:336] predicted:four one two two six eight four one four two


INFO: Epoch 8, global step 270: 'val_wer' reached 0.00776 (best 0.00776), saving model to '/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters--val_wer=0.0078-epoch=8.ckpt' as top 3
I0707 13:27:51.216756 131498730960704 rank_zero.py:64] Epoch 8, global step 270: 'val_wer' reached 0.00776 (best 0.00776), saving model to '/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters--val_wer=0.0078-epoch=8.ckpt' as top 3


[NeMo I 2024-07-07 13:27:52 nemo_model_checkpoint:217] New best .nemo model saved to: /workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters.nemo
[NeMo I 2024-07-07 13:27:53 wer:334] 
    
[NeMo I 2024-07-07 13:27:53 wer:335] reference:enter eight one nine six
[NeMo I 2024-07-07 13:27:53 wer:336] predicted:enter eight one nine six
[NeMo I 2024-07-07 13:27:54 wer:334] 
    
[NeMo I 2024-07-07 13:27:54 wer:335] reference:p i t t s b u r g h
[NeMo I 2024-07-07 13:27:54 wer:336] predicted:p i t t s b u r g h
[NeMo I 2024-07-07 13:27:54 wer:334] 
    
[NeMo I 2024-07-07 13:27:54 wer:335] reference:a r a p a h o e
[NeMo I 2024-07-07 13:27:54 wer:336] predicted:a r a p a h o e
[NeMo I 2024-07-07 13:27:55 wer:334] 
    
[NeMo I 2024-07-07 13:27:55 wer:335] reference:five two one four nine five four
[NeMo I 2024-07-07 13:27:55 wer:336] predicted:five two one four nine five four
[NeMo I 2024-07-07 13:27:56 wer:334] 
    
[

INFO: `Trainer.fit` stopped: `max_steps=300` reached.
I0707 13:27:56.647670 131498730960704 rank_zero.py:64] `Trainer.fit` stopped: `max_steps=300` reached.
INFO: Restoring states from the checkpoint path at /workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters--val_wer=0.0078-epoch=8.ckpt
I0707 13:27:57.032257 131498730960704 rank_zero.py:64] Restoring states from the checkpoint path at /workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters--val_wer=0.0078-epoch=8.ckpt
INFO: Restored all states from the checkpoint at /workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/ASR-Adapters/2024-07-07_13-24-36/checkpoints/ASR-Adapters--val_wer=0.0078-epoch=8.ckpt
I0707 13:27:57.202682 131498730960704 rank_zero.py:64] Restored all states from the checkpoint at /workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/experiments/

-----

After training, save the final checkpoint to a nemo file to evaluate. We also save just the adapter module itself, as that is much smaller than the size of the full model.

In [32]:
trainer.train_dataloader.num_workers = 0

In [34]:
model.save_to("/content/adapted_model.nemo")

In [35]:
model.save_adapters('/content/adapter_modules.pt')

# Evaluate the adapted model

Now that we have finished the adaptation step and saved a trained NeMo file, we can evaluate the accuracy of our adapted model on the test set of AN4.

## Evaluate the adapter-enabled model

In [49]:
!python scripts/speech_to_text_eval.py \
  model_path="/content/adapted_model.nemo" \
  dataset_manifest=$TEST_MANIFEST \
  output_filename="/content/adapted_predictions.json" \
  batch_size=32 \
  use_cer=False

[NeMo W 2024-07-07 13:40:21 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo I 2024-07-07 13:40:21 transcribe_speech:222] Hydra config: model_path: /content/adapted_model.nemo
    pretrained_name: null
    audio_dir: null
    dataset_manifest: datasets/an4/test_manifest.json
    channel_selector: null
    audio_key: audio_filepath
    eval_config_yaml: null
    presort_manifest: true
    output_filename: /content/adapted_predictions.json
    batch_size: 32
    num_workers: 0
    append_pred: false
    pred_name_postfix: null
    random_seed: null
    compute_timestamps: false
    preserve_alignment: false
    compute_langs: false
    cuda: null
    allow_mps: false
    amp: false
    amp_dtype: float16
    compute

-----

We could significantly improve the accuracy of this model on the AN4 dataset with a very short training schedule with a small number of parameters. 

**Note**: Since AN4 is a relatively simple dataset, the gains are very large in this example. We generally observe more modest improvements with such short training schedules on realistic datasets (but gains of this range are easily attainable with more data or precise training schedules to avoid overfitting).

Let us compare the adapted model's predictions below - 

In [50]:
print("Original")
!head -n 5 /content/unadapted_predictions.json
print("Adapted")
!head -n 5 /content/adapted_predictions.json

Original
head: cannot open '/content/unadapted_predictions.json' for reading: No such file or directory
Adapted
{"audio_filepath": "/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/datasets/an4/wav/an4test_clstk/fcaw/an406-fcaw-b.wav", "duration": 4.0, "text": "rubout g m e f three nine", "pred_text": "rubout g m e f three nine", "wer": 0.0, "tokens": 7, "ins_rate": 0.0, "del_rate": 0.0, "sub_rate": 0.0}
{"audio_filepath": "/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/datasets/an4/wav/an4test_clstk/fcaw/an407-fcaw-b.wav", "duration": 4.1, "text": "erase c q q f seven", "pred_text": "erase c q q f seven", "wer": 0.0, "tokens": 6, "ins_rate": 0.0, "del_rate": 0.0, "sub_rate": 0.0}
{"audio_filepath": "/workspace/nemo/NeMo-opensource/tutorials/asr/asr_adapters/datasets/an4/wav/an4test_clstk/fcaw/an408-fcaw-b.wav", "duration": 3.3, "text": "b a o z five three", "pred_text": "b a o z five three", "wer": 0.0, "tokens": 6, "ins_rate": 0.0, "del_rate": 0.0, "sub_rate": 0

## Evaluate the adapter-disabled model

Now, let us disable the adapters and recover the original performance of the model. We do this as a sanity test, to check that indeed the "base" model is still intact, even if adapter training has occurred.

In [51]:
model.set_enabled_adapters(enabled=False)
model.save_to("/content/adapter_disabled_model.nemo")

[NeMo I 2024-07-07 13:40:43 adapter_mixins:719] Setting adapter 'AN4' status : Enabled = False


In [52]:
!python scripts/speech_to_text_eval.py \
  model_path="/content/adapter_disabled_model.nemo" \
  dataset_manifest=$TEST_MANIFEST \
  output_filename="/content/adapter_disabled_predictions.json" \
  batch_size=32 \
  use_cer=False

[NeMo W 2024-07-07 13:40:58 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo I 2024-07-07 13:40:58 transcribe_speech:222] Hydra config: model_path: /content/adapter_disabled_model.nemo
    pretrained_name: null
    audio_dir: null
    dataset_manifest: datasets/an4/test_manifest.json
    channel_selector: null
    audio_key: audio_filepath
    eval_config_yaml: null
    presort_manifest: true
    output_filename: /content/adapter_disabled_predictions.json
    batch_size: 32
    num_workers: 0
    append_pred: false
    pred_name_postfix: null
    random_seed: null
    compute_timestamps: false
    preserve_alignment: false
    compute_langs: false
    cuda: null
    allow_mps: false
    amp: false
    amp_dtype: f

# [EXTRA] Check that accuracy can be recovered after adaptation

This is a more explicit test than simply checking the WER above - here we do sample by sample check to ensure that predicted text remains the same.

In [53]:
original_transcripts = read_manifest('/content/unadapted_predictions.json')
adapter_disabled_transcripts = read_manifest('/content/adapter_disabled_predictions.json')

for orig, new in zip(original_transcripts, adapter_disabled_transcripts):
  match = orig['pred_text'] == new['pred_text']
  if not match:
    print("Sample did not match after disabling adapter !")
    print("Original = ", orig['pred_text'])
    print("Adapters disabled = ", new['pred_text']) 
    print()

Exception: Manifest file could not be opened: <class 'nemo.utils.data_utils.DataStoreObject'>: store_path=/content/unadapted_predictions.json, local_path=/content/unadapted_predictions.json

# [EXTRA] Add as many adapters as needed

Now that we have showcased how to utilize adapters for domain adaptation, we can take this further and adapt even more datasets - as many as needed!

There is no implicit restriction on how many adapters can be added, as shown below. Still, we do recommend freezing all adapters and training only one at a time to prevent cross-interaction between adapters.

In [ ]:
model.add_adapter(name="AN4-v2", cfg=adapter_cfg)

In [ ]:
model.set_enabled_adapters(enabled=False)
model.set_enabled_adapters(name='AN4-v2', enabled=True)

model.freeze()
model.unfreeze_enabled_adapters()

model.summarize()

# Further reading

For efficient scripts to add, train, and evaluate adapter augmented models, please refer to the [Adapters example section](https://github.com/NVIDIA/NeMo/tree/main/examples/asr/asr_adapters).

Please follow the following articles that discuss the use of adapters in ASR - 
- [Exploiting Adapters for Cross-lingual Low-resource Speech Recognition](https://arxiv.org/abs/2105.11905)
- [Efficient Adapter Transfer of Self-Supervised Speech Models for Automatic Speech Recognition](https://arxiv.org/abs/2202.03218)
